In [1]:
import csv
import tqdm
import numpy as np
import pandas as pd

In [2]:
# %pip install requests urllib if needed

import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'

In [3]:
# public_key = 'https://disk.yandex.ru/d/I2h5KffMQh3Uzg'

# final_url = base_url + urlencode(dict(public_key=public_key))
# response = requests.get(final_url)
# download_url = response.json()['href']
# response = requests.get(download_url)

# dist_path = 'logs_season_1.csv.zip'
# with open(dist_path, 'wb') as f:
#     f.write(response.content)

In [4]:
public_key = 'https://disk.yandex.ru/d/LFKgJrXog_XmlQ'

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
response = requests.get(download_url)

dist_path = 'test_season_3.csv.zip'
with open(dist_path, 'wb') as f:
    f.write(response.content)

In [5]:
# public_key = 'https://disk.yandex.ru/d/GSPiacfjLNzYlA'

# final_url = base_url + urlencode(dict(public_key=public_key))
# response = requests.get(final_url)
# download_url = response.json()['href']
# response = requests.get(download_url)

# dist_path = 'logs_season_3.csv.zip'
# with open(dist_path, 'wb') as f:
#     f.write(response.content)

In [6]:
import zipfile

with zipfile.ZipFile("test_season_3.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("datasets")

In [7]:
df_3 = pd.read_csv("datasets/test_season_3.csv", sep=',')

In [8]:
df_3.head()

,Unnamed: 0,bid_id,timestamp,user_agent,region_id,city_id,domain,ad_slot_width,ad_slot_height,ad_slot_visibility,ad_slot_format,creative_id,advertiser_id,user_profile_ids,click
0,0,b8c557a16cdd9cea7fa61df79bdb392d,20131022071300438,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,216,233,5eaa6ffce5e827b0907e99679585db6b,468,60,OtherView,Na,7328,2259,"10684,14273,10052,10133",0
1,1,ce5a2b810ada0cad122508b4a9d1e2fd,20131022000104929,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,216,219,72187db00c907f6304a240c763a6e70a,300,250,FirstView,Na,7323,2259,10006,0
2,2,61fb95442aaa7cade3761b7f329a1e73,20131022185900984,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.1...,216,229,39a2330a5cbc1a0ff7104e6aaddb1f68,200,200,OtherView,Na,7319,2259,"10129,10024,10120,10145,10142,10063",0
3,3,7f9c3a72df3e4ded04870c08fcbcf72b,20131022185502830,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,216,222,3043163ba84753b4b51dd3290caeae67,300,250,Na,Na,7323,2259,10684,0
4,4,de8192350f7e0f3223c43cf0c8cfb247,20131022013103085,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,216,233,35bb34dd34f29f2b5415de994c37acff,728,90,OtherView,Na,7330,2259,"10059,10117,10075,10006,10110,16706,10127,1340...",0


In [9]:
! rm -rf datasets/test_season_3.csv

In [10]:
from datetime import datetime

def extract_time_features(timestamp):
    ts_str = str(timestamp)
    ts_format = '%Y%m%d%H%M%S'
    dt = datetime.strptime(ts_str[:14], ts_format)  # Parse the datetime
    if dt.hour < 10:
        part_of_day = 1
    elif 10 <= dt.hour < 18:
        part_of_day = 2
    else:
        part_of_day = 3

    weekday = dt.weekday()
    weekend = 1 if weekday >= 5 else 2
    return part_of_day, weekday, weekend

df_3[['part_of_day', 'weekday', 'weekend']] = df_3['timestamp'].apply(
    lambda ts: pd.Series(extract_time_features(ts))
)

df_3.head()

,Unnamed: 0,bid_id,timestamp,user_agent,region_id,city_id,domain,ad_slot_width,ad_slot_height,ad_slot_visibility,ad_slot_format,creative_id,advertiser_id,user_profile_ids,click,part_of_day,weekday,weekend
0,0,b8c557a16cdd9cea7fa61df79bdb392d,20131022071300438,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,216,233,5eaa6ffce5e827b0907e99679585db6b,468,60,OtherView,Na,7328,2259,"10684,14273,10052,10133",0,1,1,2
1,1,ce5a2b810ada0cad122508b4a9d1e2fd,20131022000104929,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,216,219,72187db00c907f6304a240c763a6e70a,300,250,FirstView,Na,7323,2259,10006,0,1,1,2
2,2,61fb95442aaa7cade3761b7f329a1e73,20131022185900984,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.1...,216,229,39a2330a5cbc1a0ff7104e6aaddb1f68,200,200,OtherView,Na,7319,2259,"10129,10024,10120,10145,10142,10063",0,3,1,2
3,3,7f9c3a72df3e4ded04870c08fcbcf72b,20131022185502830,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,216,222,3043163ba84753b4b51dd3290caeae67,300,250,Na,Na,7323,2259,10684,0,3,1,2
4,4,de8192350f7e0f3223c43cf0c8cfb247,20131022013103085,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,216,233,35bb34dd34f29f2b5415de994c37acff,728,90,OtherView,Na,7330,2259,"10059,10117,10075,10006,10110,16706,10127,1340...",0,1,1,2


In [11]:
df_3.columns

Index(['Unnamed: 0', 'bid_id', 'timestamp', 'user_agent', 'region_id',
       'city_id', 'domain', 'ad_slot_width', 'ad_slot_height',
       'ad_slot_visibility', 'ad_slot_format', 'creative_id', 'advertiser_id',
       'user_profile_ids', 'click', 'part_of_day', 'weekday', 'weekend'],
      dtype='object')

In [12]:
df_3['click'].sum()

1155

In [13]:
len(df_3)

1579086

In [14]:
f"share of clicks = {df_3['click'].sum() / len(df_3)}"

'share of clicks = 0.0007314357799385214'

In [15]:
df_3.dtypes

Unnamed: 0             int64
bid_id                object
timestamp              int64
user_agent            object
region_id              int64
city_id                int64
domain                object
ad_slot_width          int64
ad_slot_height         int64
ad_slot_visibility    object
ad_slot_format        object
creative_id            int64
advertiser_id          int64
user_profile_ids      object
click                  int64
part_of_day            int64
weekday                int64
weekend                int64
dtype: object

In [16]:
values_to_fill = {"user_agent": "", "domain": "", "ad_slot_visibility": "other", "ad_slot_format": "other"}
df_3 = df_3.fillna(value=values_to_fill)

In [17]:
oses = ["windows", "ios", "mac", "android", "linux"]
browsers = ["chrome", "sogou", "maxthon", "safari", "firefox", "theworld", "opera", "ie"]
mobile_indicators = ["mobile", "iphone", "android"]

def parse_user_agent(user_agent):
    if type(user_agent) != str:
        print(user_agent)
    ua_lower = user_agent.lower()
    detected_os = next((os for os in oses if os in ua_lower), "other")
    detected_browser = next((browser for browser in browsers if browser in ua_lower), "other")
    device_type = True if any(indicator in ua_lower for indicator in mobile_indicators) else False
    return detected_os, detected_browser, device_type

df_3[['os', 'browser', 'device_type']] = df_3['user_agent'].apply(lambda ua: pd.Series(parse_user_agent(ua)))

df_3.head()

,Unnamed: 0,bid_id,timestamp,user_agent,region_id,city_id,domain,ad_slot_width,ad_slot_height,ad_slot_visibility,ad_slot_format,creative_id,advertiser_id,user_profile_ids,click,part_of_day,weekday,weekend,os,browser,device_type
0,0,b8c557a16cdd9cea7fa61df79bdb392d,20131022071300438,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,216,233,5eaa6ffce5e827b0907e99679585db6b,468,60,OtherView,Na,7328,2259,"10684,14273,10052,10133",0,1,1,2,windows,ie,False
1,1,ce5a2b810ada0cad122508b4a9d1e2fd,20131022000104929,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,216,219,72187db00c907f6304a240c763a6e70a,300,250,FirstView,Na,7323,2259,10006,0,1,1,2,windows,ie,False
2,2,61fb95442aaa7cade3761b7f329a1e73,20131022185900984,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.1...,216,229,39a2330a5cbc1a0ff7104e6aaddb1f68,200,200,OtherView,Na,7319,2259,"10129,10024,10120,10145,10142,10063",0,3,1,2,windows,chrome,False
3,3,7f9c3a72df3e4ded04870c08fcbcf72b,20131022185502830,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,216,222,3043163ba84753b4b51dd3290caeae67,300,250,Na,Na,7323,2259,10684,0,3,1,2,windows,ie,False
4,4,de8192350f7e0f3223c43cf0c8cfb247,20131022013103085,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,216,233,35bb34dd34f29f2b5415de994c37acff,728,90,OtherView,Na,7330,2259,"10059,10117,10075,10006,10110,16706,10127,1340...",0,1,1,2,windows,chrome,False


In [18]:
values_to_fill = {"region_id": 0, "city_id": 0, "creative_id": 0, "advertiser_id": 0, "user_profile_ids": 0, }
df_3 = df_3.fillna(value=values_to_fill)

In [19]:
def calculate_screen_share(row):
    screen_area = row['ad_slot_width'] * row['ad_slot_height']
    if row['device_type'] == 1:  # mobile device
        display_area = 800 * 600
    else:  # desktop
        display_area = 1366 * 768
    return screen_area / display_area

df_3['ad_slot_screen_share'] = df_3.apply(calculate_screen_share, axis=1)
df_3.head()

,Unnamed: 0,bid_id,timestamp,user_agent,region_id,city_id,domain,ad_slot_width,ad_slot_height,ad_slot_visibility,ad_slot_format,creative_id,advertiser_id,user_profile_ids,click,part_of_day,weekday,weekend,os,browser,device_type,ad_slot_screen_share
0,0,b8c557a16cdd9cea7fa61df79bdb392d,20131022071300438,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,216,233,5eaa6ffce5e827b0907e99679585db6b,468,60,OtherView,Na,7328,2259,"10684,14273,10052,10133",0,1,1,2,windows,ie,False,0.026766
1,1,ce5a2b810ada0cad122508b4a9d1e2fd,20131022000104929,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,216,219,72187db00c907f6304a240c763a6e70a,300,250,FirstView,Na,7323,2259,10006,0,1,1,2,windows,ie,False,0.071491
2,2,61fb95442aaa7cade3761b7f329a1e73,20131022185900984,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.1...,216,229,39a2330a5cbc1a0ff7104e6aaddb1f68,200,200,OtherView,Na,7319,2259,"10129,10024,10120,10145,10142,10063",0,3,1,2,windows,chrome,False,0.038128
3,3,7f9c3a72df3e4ded04870c08fcbcf72b,20131022185502830,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,216,222,3043163ba84753b4b51dd3290caeae67,300,250,Na,Na,7323,2259,10684,0,3,1,2,windows,ie,False,0.071491
4,4,de8192350f7e0f3223c43cf0c8cfb247,20131022013103085,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,216,233,35bb34dd34f29f2b5415de994c37acff,728,90,OtherView,Na,7330,2259,"10059,10117,10075,10006,10110,16706,10127,1340...",0,1,1,2,windows,chrome,False,0.062454


In [20]:
final_df_3 = df_3.drop("timestamp", axis=1).drop("user_agent", axis=1).drop("ad_slot_width", axis=1).drop("ad_slot_height", axis=1)

In [21]:
final_df_3["is_mobile_device"] = final_df_3["device_type"]
final_df_3 = final_df_3.drop("device_type", axis=1)
final_df_3.head()


,Unnamed: 0,bid_id,region_id,city_id,domain,ad_slot_visibility,ad_slot_format,creative_id,advertiser_id,user_profile_ids,click,part_of_day,weekday,weekend,os,browser,ad_slot_screen_share,is_mobile_device
0,0,b8c557a16cdd9cea7fa61df79bdb392d,216,233,5eaa6ffce5e827b0907e99679585db6b,OtherView,Na,7328,2259,"10684,14273,10052,10133",0,1,1,2,windows,ie,0.026766,False
1,1,ce5a2b810ada0cad122508b4a9d1e2fd,216,219,72187db00c907f6304a240c763a6e70a,FirstView,Na,7323,2259,10006,0,1,1,2,windows,ie,0.071491,False
2,2,61fb95442aaa7cade3761b7f329a1e73,216,229,39a2330a5cbc1a0ff7104e6aaddb1f68,OtherView,Na,7319,2259,"10129,10024,10120,10145,10142,10063",0,3,1,2,windows,chrome,0.038128,False
3,3,7f9c3a72df3e4ded04870c08fcbcf72b,216,222,3043163ba84753b4b51dd3290caeae67,Na,Na,7323,2259,10684,0,3,1,2,windows,ie,0.071491,False
4,4,de8192350f7e0f3223c43cf0c8cfb247,216,233,35bb34dd34f29f2b5415de994c37acff,OtherView,Na,7330,2259,"10059,10117,10075,10006,10110,16706,10127,1340...",0,1,1,2,windows,chrome,0.062454,False


In [22]:
final_df_3.to_csv('parsed_test_data_no_extra_season_3.csv.zip', compression='zip')